<a href="https://colab.research.google.com/github/SaumyaRaj188/RNN_Novel_Writer/blob/main/RNN_Novel_Writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
from keras.preprocessing import sequence
import numpy as np
import os

# Data Preprocessing

In [2]:
from google.colab import files
file_path = list(files.upload().keys())[0]

Saving five_centimeters_per_second.txt to five_centimeters_per_second.txt


In [3]:
with open(file_path, 'r') as file:
    text = file.read()

print(text[:1000:])

Chapter 1: Cherry Blossom

"Hey, it looks just like snow, "Akari had said.

It was seventeen years ago when she said that. We had just become elementary six students and we would always walk together around the small grove on our way home with our schoolbags on our little backs. It was spring and a countless number of cherry blossoms were in full bloom on the trees, their petals dancing soundlessly in the air, covering the asphalt beneath our feet in a blanket of white. The air was warm and the sky hung overhead as if it was a great canvas covered with light blue paint. Not far from us ran the main road and the Odasaki railroad crossing but none of its noise seemed to reach us. Only the chirping of birds could be heard as if a blessing from spring. There was no one else was around.

It was as if it was just a painting of a certain spring scene.

That's right. At least in my memories that moment of time was like a painting. You could say they were just a collection of images. When I try

In [4]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

encoded_text = text_to_int(text)

In [5]:
print(encoded_text[:1000:])

[25 55 48 63 67 52 65  1 11 20  1 25 55 52 65 65 72  1 24 59 62 66 66 62
 60  0  0  3 30 52 72  7  1 56 67  1 59 62 62 58 66  1 57 68 66 67  1 59
 56 58 52  1 66 61 62 70  7  1  3 23 58 48 65 56  1 55 48 51  1 66 48 56
 51  9  0  0 31 67  1 70 48 66  1 66 52 69 52 61 67 52 52 61  1 72 52 48
 65 66  1 48 54 62  1 70 55 52 61  1 66 55 52  1 66 48 56 51  1 67 55 48
 67  9  1 45 52  1 55 48 51  1 57 68 66 67  1 49 52 50 62 60 52  1 52 59
 52 60 52 61 67 48 65 72  1 66 56 71  1 66 67 68 51 52 61 67 66  1 48 61
 51  1 70 52  1 70 62 68 59 51  1 48 59 70 48 72 66  1 70 48 59 58  1 67
 62 54 52 67 55 52 65  1 48 65 62 68 61 51  1 67 55 52  1 66 60 48 59 59
  1 54 65 62 69 52  1 62 61  1 62 68 65  1 70 48 72  1 55 62 60 52  1 70
 56 67 55  1 62 68 65  1 66 50 55 62 62 59 49 48 54 66  1 62 61  1 62 68
 65  1 59 56 67 67 59 52  1 49 48 50 58 66  9  1 31 67  1 70 48 66  1 66
 63 65 56 61 54  1 48 61 51  1 48  1 50 62 68 61 67 59 52 66 66  1 61 68
 60 49 52 65  1 62 53  1 50 55 52 65 65 72  1 49 59

In [6]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(encoded_text[:1000:]))

Chapter 1: Cherry Blossom

"Hey, it looks just like snow, "Akari had said.

It was seventeen years ago when she said that. We had just become elementary six students and we would always walk together around the small grove on our way home with our schoolbags on our little backs. It was spring and a countless number of cherry blossoms were in full bloom on the trees, their petals dancing soundlessly in the air, covering the asphalt beneath our feet in a blanket of white. The air was warm and the sky hung overhead as if it was a great canvas covered with light blue paint. Not far from us ran the main road and the Odasaki railroad crossing but none of its noise seemed to reach us. Only the chirping of birds could be heard as if a blessing from spring. There was no one else was around.

It was as if it was just a painting of a certain spring scene.

That's right. At least in my memories that moment of time was like a painting. You could say they were just a collection of images. When I try

In [7]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
Chapter 1: Cherry Blossom

"Hey, it looks just like snow, "Akari had said.

It was seventeen years a

OUTPUT
hapter 1: Cherry Blossom

"Hey, it looks just like snow, "Akari had said.

It was seventeen years ag


EXAMPLE

INPUT
o when she said that. We had just become elementary six students and we would always walk together a

OUTPUT
 when she said that. We had just become elementary six students and we would always walk together ar


In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Creating Model

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           18944     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 74)            75850     
                                                                 
Total params: 5341770 (20.38 MB)
Trainable params: 5341770 (20.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [14]:
model.compile(optimizer='adam', loss=loss)

In [15]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Training

In [16]:
history = model.fit(data, epochs=400, callbacks=[checkpoint_callback])

Epoch 1/400
28/28 [==============================] - 6s 94ms/step - loss: 3.1499
Epoch 2/400
28/28 [==============================] - 2s 77ms/step - loss: 2.7769
Epoch 3/400
28/28 [==============================] - 2s 74ms/step - loss: 2.4575
Epoch 4/400
28/28 [==============================] - 3s 81ms/step - loss: 2.2986
Epoch 5/400
28/28 [==============================] - 3s 74ms/step - loss: 2.1862
Epoch 6/400
28/28 [==============================] - 2s 74ms/step - loss: 2.0898
Epoch 7/400
28/28 [==============================] - 2s 73ms/step - loss: 1.9863
Epoch 8/400
28/28 [==============================] - 2s 75ms/step - loss: 1.8910
Epoch 9/400
28/28 [==============================] - 2s 76ms/step - loss: 1.8058
Epoch 10/400
28/28 [==============================] - 2s 79ms/step - loss: 1.7219
Epoch 11/400
28/28 [==============================] - 2s 75ms/step - loss: 1.6476
Epoch 12/400
28/28 [==============================] - 2s 76ms/step - loss: 1.5792
Epoch 13/400
28/28 [=====

#Loading Model

In [17]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [18]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

# Text Generation

In [19]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [20]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: His sweet voice, and his warm hands
His sweet voice, and his warm hands to his kind voice. It was always so full of emotion. Now some world answer, he got up and shook her by the shoulders.

"Hey, Sakaguchi-san! What's wrong? Are you alrigher shoulder up all the way to her long n. . . Tohno-kun. I beg you. . .

Please don't be so nice to me any more. . .

At that instant, the corridor the skirt beat time gazing out the window during all the lestralgic was too busy deciding wr the high rise buildings of Nakano, listening to the sound of the train churning along, he suddenly asked but in the time they speakers. "Attention please. Due to the delayed schedule of all trains thinjike area. The shutters of the kiosk were shut tight. It felt like I had lost my way but there were a wouther letter over and over again. Even during classes I would secretly slip it in bet
